In [16]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
from pprint import pprint

#BITCOIN API: url setup
url = 'https://api.coindesk.com/v1/bpi/historical/close.json?'
default = 'start=2020-01-01&end=2020-12-31'

# set start and end date variables
# NOTE: must stay within 2020 range, csv files (dow & gold) are limited to this range
start_date = '2020-05-25'
end_date = '2020-06-05'

# DOW: file to Load, read csv, dataframe
file_to_load = "data/dow.csv"
dow_data = pd.read_csv(file_to_load)
dow_data_df = pd.DataFrame(dow_data)

# GOLD: file to load, read csv, dataframe
file_to_load = "data/gold.csv"
gold_data = pd.read_csv(file_to_load)
gold_data_df = pd.DataFrame(gold_data)
gold_data_df.columns = gold_data_df.columns.tolist()[:1] + ['Close'] + gold_data_df.columns.tolist()[2:]


In [17]:
query_url = f'{url}start={start_date}&end={end_date}'
pprint(requests.get(query_url).json())

{'bpi': {'2020-05-25': 8776.34,
         '2020-05-26': 8871.05,
         '2020-05-27': 8923.05,
         '2020-05-28': 9207.95,
         '2020-05-29': 9448.365,
         '2020-05-30': 9550.815,
         '2020-05-31': 9545.15,
         '2020-06-01': 9564.95,
         '2020-06-02': 10136.5,
         '2020-06-03': 9594.45,
         '2020-06-04': 9529.55,
         '2020-06-05': 9836.805},
 'disclaimer': 'This data was produced from the CoinDesk Bitcoin Price Index. '
               'BPI value data returned as USD.',
 'time': {'updated': 'Jun 6, 2020 00:03:00 UTC',
          'updatedISO': '2020-06-06T00:03:00+00:00'}}


In [39]:
response = requests.get(query_url).json()
my_dict = response['bpi']
btc_df = pd.DataFrame(list(my_dict.items()),columns = ['Close Date','Close Price']) 
btc_df

,Close Date,Close Price
0,2020-05-25,8776.340
1,2020-05-26,8871.050
2,2020-05-27,8923.050
3,2020-05-28,9207.950
4,2020-05-29,9448.365
5,2020-05-30,9550.815
6,2020-05-31,9545.150
7,2020-06-01,9564.950
8,2020-06-02,10136.500
9,2020-06-03,9594.450


In [40]:
dow_data_df['Date'] = pd.to_datetime(dow_data_df['Date'])

dow_clean = dow_data_df.loc[(dow_data_df['Date'] >= start_date) & (dow_data_df['Date'] <= end_date)]
dow_clean = dow_clean[['Date', 'Close']]

dow_df = dow_clean.rename(columns={'Date':'Close Date','Close':'Close Price'})
dow_df

,Close Date,Close Price
100,2020-05-26,24995.10938
101,2020-05-27,25548.26953
102,2020-05-28,25400.64063
103,2020-05-29,25383.10938
104,2020-06-01,25475.01953
105,2020-06-02,25742.65039
106,2020-06-03,26269.89063
107,2020-06-04,26281.82031
108,2020-06-05,27110.98047


In [36]:
gold_data_df['Date'] = pd.to_datetime(gold_data_df['Date'])

# clean data - take out out-of-range data based on start_/end_date variables above
# return only date and close price columns
gold_clean = gold_data_df.loc[(gold_data_df['Date'] >= start_date) & (gold_data_df['Date'] <= end_date)]
gold_clean = gold_clean.sort_values(by=['Date']).reset_index(drop=True)
gold_clean = gold_clean.iloc[:,0:2]

# rename date column to prep for merge
gold_df = gold_cleaned.rename(columns={'Date':'Close Date','Close':'Close Price'})

gold_df

,Close Date,Close Price
0,2020-05-26,1728.2
1,2020-05-27,1726.8
2,2020-05-28,1728.3
3,2020-05-29,1736.9
4,2020-06-01,1750.3
5,2020-06-02,1734.0
6,2020-06-03,1704.8
7,2020-06-04,1727.4
8,2020-06-05,1683.0


In [43]:
merged = pd.merge(btc_df, dow_df, on='Close Date', how='left')
merged['Close Date'] = pd.to_datetime(merged['Close Date'])



ValueError: You are trying to merge on object and datetime64[ns] columns. If you wish to proceed you should use pd.concat